#### 実行環境はGoogle Colabolatory

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! nvidia-smi

Sat Nov 27 14:00:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install albumentations==0.4.5
!pip install timm

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import albumentations as A
import cv2
import timm
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
from glob import glob

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error

In [5]:
# カレントディレクトリの指定
%cd /content/drive/MyDrive/分析コンペ/05_ProbSpace/Kiva／クラウドファンディングの資金調達額予測

# 各種設定
DICT_DTYPE = {'IMAGE_ID': 'str'}
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
COL_IMAGE_ID = 'IMAGE_ID'
COL_IMAGE_PATH = 'IMAGE_PATH'
COL_LOAN_AMOUNT = 'LOAN_AMOUNT'
COL_FOLD = 'FOLD'
IMAGE_SIZE = 224
MODEL_NAME = 'tf_efficientnet_b0_ns'
TARGET_SIZE = 1
BATCH_SIZE = 32
FOLDS = 5

/content/drive/MyDrive/分析コンペ/05_ProbSpace/Kiva／クラウドファンディングの資金調達額予測


In [6]:
# 乱数シードの固定
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [7]:
# IMAGE_IDにあって、画像データにないIDの特定
def no_exist_image_id(df, image_path_list):
    global COL_IMAGE_ID
    list_image_id = []
    for image_path in image_path_list:
        image_id = image_path[18:25]
        list_image_id.append(image_id)
    list_no_exist_image_id = set(df[COL_IMAGE_ID]) - set(list_image_id)
    return list_no_exist_image_id

In [8]:
# kfold
def get_kfold(df, n_splits):
    sk = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
    df[COL_FOLD] = -1
    for i, (train_idx, valid_idx) in enumerate(sk.split(df, df[COL_LOAN_AMOUNT]), start=1):
        df.loc[valid_idx, COL_FOLD] = i
    return df

In [10]:
# データの読み込み(使用する列のみの読み込み)
df_train = pd.read_csv('data/train.csv', usecols=[COL_IMAGE_ID, COL_LOAN_AMOUNT], dtype=DICT_DTYPE)
df_test = pd.read_csv('data/test.csv', usecols=[COL_IMAGE_ID], dtype=DICT_DTYPE)
df_image_path_train = pd.read_csv('data/train_image_path.csv')
df_image_path_test = pd.read_csv('data/test_image_path.csv')
LIST_IMAGE_PATH_TRAIN = df_image_path_train['TRAIN_IMAGE_PATH'].values.tolist()
LIST_IMAGE_PATH_TEST = df_image_path_test['TEST_IMAGE_PATH'].values.tolist()

In [17]:
# 画像IDにあって、画像がないIDをリストで取得
list_no_exist_id_train = list(no_exist_image_id(df=df_train, image_path_list=LIST_IMAGE_PATH_TRAIN))
# 画像がない画像IDを欠損値に変換
df_train[COL_IMAGE_ID].replace(list_no_exist_id_train, np.nan, inplace=True)
# 欠損値の削除
df_train = df_train.dropna().reset_index(drop=True)
# 画像のパスを新しい列をして追加
df_train[COL_IMAGE_PATH] = 'data/train_images/' + df_train['IMAGE_ID'] + '.jpg'
df_test[COL_IMAGE_PATH] = 'data/test_images/' + df_test['IMAGE_ID'] + '.jpg'
# Stratified_Kfold
df_train = get_kfold(df=df_train, n_splits=5)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


In [19]:
class TrainDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['IMAGE_PATH'].values
        self.labels = df['LOAN_AMOUNT'].values
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_path = self.file_names[idx]
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = torch.tensor(self.labels[idx]).float()
        if self.transform:
            image = self.transform(image=image)['image']
        return image, label

class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['IMAGE_PATH'].values
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_path = self.file_names[idx]
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image=image)['image']
        return image

In [20]:
def get_transforms(*, data):
    
    if data == 'train':
        return A.Compose([
            A.Resize(IMAGE_SIZE, IMAGE_SIZE),
            A.HorizontalFlip(p=0.5),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

    elif data == 'valid':
        return A.Compose([
            A.Resize(IMAGE_SIZE, IMAGE_SIZE),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

In [21]:
class CNNModel(nn.Module):
    def __init__(self, model_name, target_size, pretrained=False):
        super().__init__()
        self.model_name = model_name
        self.target_size = target_size
        self.model = timm.create_model(self.model_name, pretrained=pretrained)
        self.n_features = self.model.classifier.in_features
        self.model.classifier = nn.Identity()
        self.fc = nn.Linear(self.n_features, self.target_size)

    def feature(self, image):
        feature = self.model(image)
        return feature
        
    def forward(self, image):
        feature = self.feature(image)
        output = self.fc(feature)
        return output

In [22]:
def train_fn(train_loader, model, criterion, optimizer, device):
    model.train()
    train_loss = 0.0
    train_mae = 0.0

    for images, labels in tqdm(train_loader):
        optimizer.zero_grad()
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        outputs = torch.squeeze(outputs)
        loss = criterion(outputs, labels)
        preds = outputs.detach().cpu().numpy()
        labels = labels.detach().cpu().numpy()

        train_loss += loss.item()
        train_mae += mean_absolute_error(labels, preds)
        loss.backward()
        optimizer.step()

    train_loss /= len(train_loader)
    train_mae /= len(train_loader)

    return train_loss, train_mae

In [23]:
def valid_fn(valid_loader, model, criterion, device):
    model.eval()
    valid_loss = 0.0
    valid_mae = 0.0
    valid_preds = []

    for images, labels in tqdm(valid_loader):
        images = images.to(device)
        labels = labels.to(device)
        with torch.no_grad():
            outputs = model(images)
            outputs = torch.squeeze(outputs)
            loss = criterion(outputs, labels)
            preds = outputs.detach().cpu().numpy()
            labels = labels.detach().cpu().numpy()

        valid_loss += loss.item()
        valid_mae += mean_absolute_error(labels, preds)
        valid_preds.append(preds)

    valid_loss /= len(valid_loader)
    valid_mae /= len(valid_loader)
    valid_preds = np.concatenate(valid_preds)

    return valid_loss, valid_mae, valid_preds

In [31]:
for i in range(1, FOLDS):
    model = CNNModel(model_name=MODEL_NAME, target_size=TARGET_SIZE)
    model.to(DEVICE)
    criterion = nn.L1Loss()
    optimizer = optim.Adam(params=model.parameters(), lr=0.001)
    print(f'fold: {i}')
    print('='*50)
    train = df_train[df_train['FOLD']!=i]
    train = train.sample(n=10000)
    valid = df_train[df_train['FOLD']==i]
    valid = valid.sample(n=10000)
    train_dataset = TrainDataset(train, transform=get_transforms(data='train'))
    valid_dataset = TrainDataset(valid, transform=get_transforms(data='valid'))
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=4, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, num_workers=4, shuffle=False)

    for epoch in range(5):
        train_loss, train_mae = train_fn(train_loader, model, criterion, optimizer, DEVICE)
        valid_loss, valid_mae, valid_preds = valid_fn(valid_loader, model, criterion, DEVICE)
        print(f'train_loss: {train_loss:.4f}, train_mae: {train_mae:.4f}')
        print(f'valid_loss: {valid_loss:.4f}, valid_mae: {valid_mae:.4f}')

    model_path = f'model/{i}fold_model.pth'
    torch.save(model.to('cpu').state_dict(), model_path)

    del model, criterion, optimizer

fold: 1


100%|██████████| 313/313 [13:03<00:00,  2.50s/it]


train_loss: 645.8501, train_mae: 645.8501
valid_loss: 575.3080, valid_mae: 575.3080


100%|██████████| 313/313 [00:26<00:00, 11.60it/s]


train_loss: 503.1773, train_mae: 503.1773
valid_loss: 484.1779, valid_mae: 484.1779


...

train_loss: 496.5772, train_mae: 496.5772
valid_loss: 486.1814, valid_mae: 486.1814


100%|██████████| 313/313 [00:27<00:00, 11.55it/s]


train_loss: 494.8392, train_mae: 494.8392
valid_loss: 486.8617, valid_mae: 486.8617
fold: 2


  4%|▍         | 12/313 [00:25<10:49,  2.16s/it]


KeyboardInterrupt: ignored